In [1]:

import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import joblib

In [2]:
df = pd.read_csv('/content/PB_All_2000_2021 (2).csv', sep=';')

In [3]:
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')

In [4]:
df = df.sort_values(by=['id', 'date'])

In [5]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [7]:
numeric_cols = ['NH4', 'BSK5', 'Suspended', 'O2', 'NO3', 'NO2', 'SO4', 'PO4', 'CL']
for col in numeric_cols:
    df[col].fillna(df[col].median(), inplace=True)

/tmp/ipython-input-7-3882443957.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


In [8]:
for col in numeric_cols:
    lower_bound = df[col].quantile(0.01)
    upper_bound = df[col].quantile(0.99)
    df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)

In [9]:
pollutants = ['O2', 'NO3', 'NO2', 'SO4', 'PO4', 'CL']
for pollutant in pollutants:
    df[f'{pollutant}_lag1'] = df.groupby('id')[pollutant].shift(1)

In [10]:
df = df.dropna()

In [11]:
features = ['id', 'year', 'month', 'NH4', 'BSK5', 'Suspended'] + [f'{p}_lag1' for p in pollutants]
X = df[features]
y = df[pollutants]

In [12]:
X_encoded = pd.get_dummies(X, columns=['id'], drop_first=True)
scaler = StandardScaler()
X_encoded[['year', 'month', 'NH4', 'BSK5', 'Suspended'] + [f'{p}_lag1' for p in pollutants]] = scaler.fit_transform(
    X_encoded[['year', 'month', 'NH4', 'BSK5', 'Suspended'] + [f'{p}_lag1' for p in pollutants]]
)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [14]:
rf_base = RandomForestRegressor(random_state=42)
param_grid = {
    'estimator__n_estimators': [100, 200],
    'estimator__max_depth': [10, 20, None],
    'estimator__min_samples_split': [2, 5],
}
model_rf = MultiOutputRegressor(rf_base)
grid_search_rf = GridSearchCV(model_rf, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42)),
             n_jobs=-1,
             param_grid={'estimator__max_depth': [10, 20, None],
                         'estimator__min_samples_split': [2, 5],
                         'estimator__n_estimators': [100, 200]},
             scoring='r2')

In [15]:
best_rf = grid_search_rf.best_estimator_
print("Best Random Forest Parameters:", grid_search_rf.best_params_)

Best Random Forest Parameters: {'estimator__max_depth': 20, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 200}


In [16]:
y_pred_rf = best_rf.predict(X_test)
print("\nRandom Forest Performance on Test Data:")
for i, pollutant in enumerate(pollutants):
    mse = mean_squared_error(y_test.iloc[:, i], y_pred_rf[:, i])
    mae = mean_absolute_error(y_test.iloc[:, i], y_pred_rf[:, i])
    r2 = r2_score(y_test.iloc[:, i], y_pred_rf[:, i])
    print(f"{pollutant}:")
    print(f"   MSE: {mse:.4f}")
    print(f"   MAE: {mae:.4f}")
    print(f"   R2: {r2:.4f}\n")


Random Forest Performance on Test Data:
O2:
   MSE: 6.1433
   MAE: 1.8217
   R2: 0.5053

NO3:
   MSE: 7.0871
   MAE: 1.6521
   R2: 0.5317

NO2:
   MSE: 0.1112
   MAE: 0.1270
   R2: 0.4167

SO4:
   MSE: 319.2760
   MAE: 11.6430
   R2: 0.8954

PO4:
   MSE: 0.1130
   MAE: 0.1938
   R2: 0.4620

CL:
   MSE: 2561.7103
   MAE: 12.2194
   R2: 0.9630



In [17]:
feature_importance = np.mean([est.feature_importances_ for est in best_rf.estimators_], axis=0)
importance_df = pd.DataFrame({'Feature': X_encoded.columns, 'Importance': feature_importance})
print("\nFeature Importance (Random Forest):")
print(importance_df.sort_values(by='Importance', ascending=False))


Feature Importance (Random Forest):
      Feature  Importance
10    CL_lag1    0.302910
2         NH4    0.099624
1       month    0.078076
6    NO3_lag1    0.071297
8    SO4_lag1    0.069934
4   Suspended    0.069824
5     O2_lag1    0.060380
7    NO2_lag1    0.060190
9    PO4_lag1    0.057543
0        year    0.044428
3        BSK5    0.043969
28      id_19    0.010319
22      id_13    0.006009
26      id_17    0.003908
27      id_18    0.003407
24      id_15    0.001735
31      id_22    0.001574
30      id_21    0.001568
21      id_12    0.001557
29      id_20    0.001509
25      id_16    0.001461
19      id_10    0.001251
11       id_2    0.001130
16       id_7    0.001072
23      id_14    0.001043
15       id_6    0.000987
13       id_4    0.000940
17       id_8    0.000700
12       id_3    0.000670
18       id_9    0.000558
20      id_11    0.000351
14       id_5    0.000075


In [18]:
xgb_base = XGBRegressor(random_state=42)
model_xgb = MultiOutputRegressor(xgb_base)
model_xgb.fit(X_train, y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))

In [19]:
y_pred_xgb = model_xgb.predict(X_test)
print("\nXGBoost Performance on Test Data:")
for i, pollutant in enumerate(pollutants):
    mse = mean_squared_error(y_test.iloc[:, i], y_pred_xgb[:, i])
    mae = mean_absolute_error(y_test.iloc[:, i], y_pred_xgb[:, i])
    r2 = r2_score(y_test.iloc[:, i], y_pred_xgb[:, i])
    print(f"{pollutant}:")
    print(f"   MSE: {mse:.4f}")
    print(f"   MAE: {mae:.4f}")
    print(f"   R2: {r2:.4f}\n")


XGBoost Performance on Test Data:
O2:
   MSE: 6.2464
   MAE: 1.8707
   R2: 0.4970

NO3:
   MSE: 6.9277
   MAE: 1.6596
   R2: 0.5423

NO2:
   MSE: 0.1186
   MAE: 0.1303
   R2: 0.3778

SO4:
   MSE: 336.9353
   MAE: 11.6897
   R2: 0.8896

PO4:
   MSE: 0.1296
   MAE: 0.2059
   R2: 0.3828

CL:
   MSE: 3887.2339
   MAE: 13.2355
   R2: 0.9439



In [20]:
cv_scores_rf = cross_val_score(best_rf, X_encoded, y, cv=5, scoring='r2', n_jobs=-1)
print("\nRandom Forest Cross-Validation R2 Scores:", cv_scores_rf)
print("Mean CV R2 Score:", cv_scores_rf.mean())


Random Forest Cross-Validation R2 Scores: [-0.01653818 -0.65432943 -0.38975698  0.34228207  0.3797198 ]
Mean CV R2 Score: -0.0677245458854462


In [ ]:
v_scores_rf = cross_val_score(best_rf, X_encoded, y, cv=5, scoring='r2', n_jobs=-1)
print("\nRandom Forest Cross-Validation R2 Scores:", cv_scores_rf)
print("Mean CV R2 Score:", cv_scores_rf.mean())


In [22]:
station_id = '22'
year_input = 2024
month_input = 6  # Example month (June)
# Use median values for other features and lagged features
input_data = pd.DataFrame({
    'id': [station_id], # Add station_id to input_data
    'year': [year_input],
    'month': [month_input],
    'NH4': [df['NH4'].median()],
    'BSK5': [df['BSK5'].median()],
    'Suspended': [df['Suspended'].median()],
    **{f'{p}_lag1': [df[p].median()] for p in pollutants}
})
input_encoded = pd.get_dummies(input_data, columns=['id'])
# Add station id
# input_encoded[f'id_{station_id}'] = 1 # This line is no longer needed as id is included in get_dummies
# Align with training feature columns
missing_cols = set(X_encoded.columns) - set(input_encoded.columns)
for col in missing_cols:
    input_encoded[col] = 0
input_encoded = input_encoded[X_encoded.columns]
# Scale numerical features
input_encoded[['year', 'month', 'NH4', 'BSK5', 'Suspended'] + [f'{p}_lag1' for p in pollutants]] = scaler.transform(
    input_encoded[['year', 'month', 'NH4', 'BSK5', 'Suspended'] + [f'{p}_lag1' for p in pollutants]]
)

# Predict with Random Forest
predicted_pollutants = best_rf.predict(input_encoded)[0]
print(f"\nPredicted pollutant levels for station '{station_id}' in {year_input}, month {month_input} (Random Forest):")
for p, val in zip(pollutants, predicted_pollutants):
    print(f"  {p}: {val:.2f}")

# Save the model, scaler, and column structure
joblib.dump(best_rf, 'pollution_model_rf.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(X_encoded.columns.tolist(), 'model_columns.pkl')
print("\nModel, scaler, and column structure saved!")


Predicted pollutant levels for station '22' in 2024, month 6 (Random Forest):
  O2: 8.70
  NO3: 0.98
  NO2: 0.07
  SO4: 52.86
  PO4: 0.43
  CL: 36.47

Model, scaler, and column structure saved!
